In [30]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
# from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
# from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from lazypredict.Supervised import LazyRegressor

from sklearn.linear_model import LinearRegression


In [31]:
df = pd.read_csv("csgo.csv")
df.info()
# df = df.drop(['team_a_rounds', 'team_b_rounds'], axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1133 entries, 0 to 1132
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   map            1133 non-null   object 
 1   day            1133 non-null   float64
 2   month          1133 non-null   float64
 3   year           1133 non-null   float64
 4   date           1133 non-null   object 
 5   wait_time_s    1133 non-null   float64
 6   match_time_s   1133 non-null   float64
 7   team_a_rounds  1133 non-null   float64
 8   team_b_rounds  1133 non-null   float64
 9   ping           1133 non-null   float64
 10  kills          1133 non-null   float64
 11  assists        1133 non-null   float64
 12  deaths         1133 non-null   float64
 13  mvps           1133 non-null   float64
 14  hs_percent     1133 non-null   float64
 15  points         1133 non-null   float64
 16  result         1133 non-null   object 
dtypes: float64(14), object(3)
memory usage: 150.6+ KB


In [32]:
target = 'points'
df = df.drop(['map', 'day', 'month','year','date', 'ping','team_a_rounds', 'team_b_rounds'], axis=1)
x = df.drop(target, axis=1)
y = df[[target]]
print(y)


      points
0      45.00
1      40.00
2      37.00
3      30.00
4      83.00
...      ...
1128   26.00
1129   51.00
1130   30.00
1131   31.00
1132   23.00

[1133 rows x 1 columns]


In [33]:
# result_catetories = ['Tie', 'Lost', 'Win']
# ord_transformer = OrdinalEncoder(categories=[result_catetories])
# num_transformer = StandardScaler()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [43]:
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
models, predictions = reg.fit(x_train, x_test, y_train, y_test)



 24%|██▍       | 10/42 [00:00<00:01, 19.12it/s]

GammaRegressor model failed to execute
Some value(s) of y are out of the valid range of the loss 'HalfGammaLoss'.


100%|██████████| 42/42 [00:03<00:00, 12.99it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000060 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 580
[LightGBM] [Info] Number of data points in the train set: 906, number of used features: 10
[LightGBM] [Info] Start training from score 39.285872


In [53]:
print(models)

                               Adjusted R-Squared  R-Squared  RMSE  Time Taken
Model                                                                         
LassoCV                                      0.94       0.94  3.69        0.07
LassoLarsIC                                  0.94       0.94  3.69        0.02
LassoLarsCV                                  0.94       0.94  3.69        0.02
LarsCV                                       0.94       0.94  3.69        0.04
OrthogonalMatchingPursuitCV                  0.94       0.94  3.69        0.02
Lars                                         0.94       0.94  3.70        0.03
LinearRegression                             0.94       0.94  3.70        0.01
TransformedTargetRegressor                   0.94       0.94  3.70        0.01
BayesianRidge                                0.94       0.94  3.70        0.01
RidgeCV                                      0.94       0.94  3.70        0.03
Ridge                                        0.94   

In [38]:
result_catetories = ['Tie', 'Lost', 'Win']
ord_transformer = OrdinalEncoder(categories=[result_catetories])
num_transformer = StandardScaler()

column_list_full = df.columns.tolist()
print(column_list_full)
column_list_full.remove('result')
column_list_full.remove('points')
preprocessor = ColumnTransformer(transformers=[
    ('num_transformer', num_transformer, column_list_full),
    ('ord_transformer', ord_transformer, ['result'])
])

['wait_time_s', 'match_time_s', 'kills', 'assists', 'deaths', 'mvps', 'hs_percent', 'points', 'result']


In [40]:
model = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('regressor', LinearRegression())
])

model.fit(x_train, y_train)

y_predict = model.predict(x_test)
print(f"Mean absolute error: {mean_absolute_error(y_test, y_predict)}")
print(f"Mean square error: {mean_squared_error(y_test, y_predict)}")
print(f"R square error: {r2_score(y_test, y_predict)}")

Mean absolute error: 2.955943830774882
Mean square error: 13.720205147953097
R square error: 0.9439096596705551
